# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL
from pyspark.sql import types as T

# Crie uma instância da classe AlinharETL

In [2]:
bucket = "bronze"
datamart = "dadosabertosreceita"
extrator_bronze = AlinharETL(bucket,datamart)

2024-10-15 10:17:09,276 - INFO - Iniciando Sessão Spark.


# Parâmetros para leitura do arquivo

In [3]:
schema = T.StructType([
    T.StructField("codigo", T.StringType(), True),
    T.StructField("descricao", T.StringType(), True),
])

In [4]:
ano_mes = extrator_bronze.encontrar_ano_mes("dadosabertosreceita")
local = f"s3a://landing/dadosabertosreceita/{ano_mes}/motivos/*.csv"

In [5]:
config_adicional = {
    'sep': ';',
    'encoding': 'latin1'
}

# Leitura do arquivo

In [6]:
df_compliance_receita_motivos = extrator_bronze.criar_view_temporaria_arquivo(local, 'compliance_motivos',schema,**config_adicional)

2024-10-15 10:17:14,103 - INFO - Lista de yaml carregada com sucesso
2024-10-15 10:17:17,611 - INFO - Criando view temporaria.


# Gravação no datalake

In [7]:
extrator_bronze.caminho_tabela_delta = 's3a://bronze/dadosabertosreceita/motivos'

In [8]:
extrator_bronze.salvar_delta(df_compliance_receita_motivos, 'overwrite')

2024-10-15 10:17:17,623 - INFO - Aguarde... Persistindo dados (overwrite)
2024-10-15 10:17:36,998 - INFO - Dados persistidos com sucesso
2024-10-15 10:17:37,000 - INFO - s3a://bronze/dadosabertosreceita/motivos
2024-10-15 10:17:37,000 - INFO - Aguarde... Realizando optimize
2024-10-15 10:17:47,520 - INFO - Optimize executado com sucesso.
2024-10-15 10:17:47,521 - INFO - Aguarde... Realizando vacuum
2024-10-15 10:18:06,959 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [9]:
extrator_bronze.parar_sessao()

2024-10-15 10:18:07,433 - INFO - Sessão Spark finalizada.
